In [ ]:
import rospy
from sensor_msgs.msg import PointCloud2, Image
import sensor_msgs.point_cloud2 as pc2
from velodyne_msgs.msg import VelodyneScan
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
from cv_bridge import CvBridge
import time
import cv2
import open3d as o3d

In [ ]:
ZED_TOPIC = "/islam/zed_pts"
VLP_TOPIC = "/islam/vlp_pts"
ZED_DEPTH = '/zed/zed_node/depth/depth_registered'

In [ ]:
VELODYNE_INDEXDER = {
    0:15,
    2:14,
    4:13,
    6:12,
    8:11,
    10:10,
    12:9,
    14:8,
    1:7,
    3:6,
    5:5,
    7:4,
    9:3,
    11:2,
    13:1,
    15:0,
}
ZED_V = 360
ZED_H = 640
LiDAR_V = 16

In [ ]:
rospy.init_node('sf', anonymous=True)
# zed_s = rospy.wait_for_message(ZED_TOPIC, PointCloud2)
# vlp_s = rospy.Subscriber(VLP_TOPIC, PointCloud2, vlp_callback)

In [ ]:
def depth_to_sph_pts(depth):
    # Make sure input depth array is a CuPy array
    depth = cp.array(depth)
    
    # get the shape of the input array
    m, n = depth.shape
    azimuth_const = 100/n
    polar_const = 30/m

    # Create a grid of row and col indices
    row_indices, col_indices = cp.meshgrid(cp.arange(m), cp.arange(n), indexing='ij')

    # Calculate polar and azimuth angles
    polar_angles = row_indices * polar_const
    azimuth_angles = col_indices * azimuth_const

    # Stack the depth, polar_angles, and azimuth_angles along the last dimension
    pts = cp.stack((depth, polar_angles, azimuth_angles), axis=-1)

    # Reshape the pts array to the desired output shape (m * n, 3)
    pts = pts.reshape(m * n, 3)

    return pts

def sph_pcd_to_cart_pcd(sph_pcd):
    sph_pcd[:,1] = cp.radians(sph_pcd[:,1])
    sph_pcd[:,2] = cp.radians(sph_pcd[:,2])

    # Convert spherical coordinates to Cartesian coordinates
    x = sph_pcd[:, 0] * cp.cos(sph_pcd[:, 1]) * cp.cos(sph_pcd[:, 2])
    y = sph_pcd[:, 0] * cp.cos(sph_pcd[:, 1]) * cp.sin(sph_pcd[:, 2])
    z = sph_pcd[:, 0] * cp.sin(sph_pcd[:, 1])
    
    cart_pts = cp.asarray([x, y, z]).T

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cart_pts.get())
    return pcd

def appendSpherical_np(xyz):
    ptsnew = np.hstack((xyz, np.zeros(xyz.shape)))
    xy = xyz[:,2]**2 + xyz[:,1]**2
    ptsnew[:,3] = np.sqrt(xy + xyz[:,0]**2)
    # ptsnew[:,4] = np.arctan2(np.sqrt(xy), xyz[:,0]) # for elevation angle defined from Z-axis down
    ptsnew[:,4] = np.arctan2(xyz[:,0], np.sqrt(xy)) # for elevation angle defined from XY-plane up
    ptsnew[:,5] = np.arctan2(xyz[:,1], xyz[:,2])
    return ptsnew

def convert_to_spherical(cloud):
    # Convert to CuPy array
    cloud = cp.asarray(cloud)

    # Convert to spherical coordinates
    x, y, z = cloud[:, 0], cloud[:, 1], cloud[:, 2]
    r = cp.sqrt(x**2 + y**2 + z**2)
    theta = cp.arctan(z / cp.sqrt(x**2 + y**2))
    phi = cp.arctan(y / x)

    return cp.column_stack((r, cp.degrees(theta), cp.degrees(phi)))
    # return appendSpherical_np(cloud)[:,3:6]

def msg2pts(msg):
    return np.array(list(pc2.read_points(msg, field_names=("x", "y", "z"))))

In [ ]:
# zed_pts = rospy.wait_for_message(ZED_TOPIC, PointCloud2)
zed_img = rospy.wait_for_message(ZED_DEPTH, Image)
vlp_pts = rospy.wait_for_message(VLP_TOPIC, PointCloud2)
if zed_img and vlp_pts:
    print('OK')

In [ ]:
vlp_pts = msg2pts(vlp_pts)

In [ ]:
vlp_pts[vlp_pts[:,1] > 0]

In [ ]:
vlp_pcd_p = o3d.geometry.PointCloud()
vlp_pcd_p.points = o3d.utility.Vector3dVector(vlp_pts[vlp_pts[:,0] > 0])

vlp_pcd_n = o3d.geometry.PointCloud()
vlp_pcd_n.points = o3d.utility.Vector3dVector(vlp_pts[vlp_pts[:,0] < 0])

o3d.visualization.draw_geometries([
    vlp_pcd_p,
    # vlp_pcd_n
    ])

In [ ]:
vlp_sph_pts = convert_to_spherical(vlp_pts[vlp_pts[:,0] > 0]).get()
print(min(vlp_sph_pts[:,2]), max(vlp_sph_pts[:,2]))


In [ ]:
# gt_mean = vlp_sph_pts[:, 2].mean()
# vlp_sph_pts[:, 0] = np.array(list(map(VELODYNE_INDEXDER.get, vlp_sph_pts[:, 0])))
mid_cutoff = 180
vlp_sph_pts[:, 2] += mid_cutoff

mask = vlp_sph_pts[:, 2] > 2*mid_cutoff

vlp_sph_pts[mask, 2] -= 2*mid_cutoff


In [ ]:
np.count_nonzero(mask)
print(min(vlp_sph_pts[:,2]), max(vlp_sph_pts[:,2]))


In [ ]:

# vlp_pcd = o3d.geometry.PointCloud()
# vlp_pcd.points = o3d.utility.Vector3dVector(msg2pts(vlp_pts))

# vlp_sph_pcd = o3d.geometry.PointCloud()
# vlp_sph_pcd.points = o3d.utility.Vector3dVector(vlp_sph_pts)

origin_pcd = o3d.geometry.PointCloud()
origin_pcd.points = o3d.utility.Vector3dVector([[0,0,0]])
origin_pcd.colors = o3d.utility.Vector3dVector(np.array([[255,0,0]]))

vlp_pcd_1 = sph_pcd_to_cart_pcd(cp.array(vlp_sph_pts))

o3d.visualization.draw_geometries([
    origin_pcd,
    # vlp_sph_pcd,
    # vlp_pcd,
    vlp_pcd_1
    ])

In [ ]:

# Split the array based on the range of the first column
vlp_sph_pts_rows = []
for i in range(LiDAR_V):
    lower = i
    upper = i + 1
    mask = (vlp_sph_pts[:, 0] >= lower) & (vlp_sph_pts[:, 0] < upper)
    sub_array = vlp_sph_pts[mask, 1:]
    vlp_sph_pts_rows.append(sub_array)

# Convert each sub-array to a 1D array sorted by the first column and using the second column as the value
vlp_depth = []
for sub_array in vlp_sph_pts_rows:
    indices = np.argsort(sub_array[:, 0])
    sorted_array = sub_array[indices, 1].tolist()

    row_len = len(sorted_array)
    num_zeros = ZED_H - row_len
    step_size = row_len // (num_zeros + 1)

    # Loop over the array and insert zeros at regular intervals
    for i in range(num_zeros):
        index = (i + 1) * step_size
        sorted_array.insert(index, gt_mean)

    vlp_depth.append(sorted_array)
vlp_depth = np.asarray(vlp_depth)

In [ ]:
plt.imshow(vlp_depth)
vlp_depth.shape

In [ ]:
bridge = CvBridge()
zed_depth = np.array(bridge.imgmsg_to_cv2(zed_img, "32FC1"))

In [ ]:
plt.imshow(zed_depth)
zed_depth.shape

In [ ]:
def lpf(img, ncutoff):
    # Apply 2D FFT to the image
    f = cp.fft.fft2(img)

    # Shift the zero frequency component to the center of the spectrum
    fshift = cp.fft.fftshift(f)

    # Create a circular mask of the same size as the spectrum
    rows, cols = img.shape
    crow, ccol = rows // 2, cols // 2
    mask = np.zeros((rows, cols), np.uint8)
    cutoff = int(min(crow, ccol)*ncutoff)
    cv2.circle(mask, (ccol, crow), cutoff, 1, -1)
    # cv2.ellipse(mask, (ccol, crow), (1, 2) * cutoff, 0, 0, 360,  1, -1)

    mask = cp.asarray(mask)

    # Apply the mask to the shifted spectrum
    fshift_filtered = fshift * mask

    # Shift the zero frequency component back to the corner of the spectrum
    f_filtered = cp.fft.ifftshift(fshift_filtered)

    # Apply the inverse 2D FFT to the filtered spectrum
    img_filtered = cp.fft.ifft2(f_filtered)
    img_filtered = cp.real(img_filtered)

    return img_filtered


def pg(input, us_rate, ncutoff, threshold=100):
    ncutoff = ncutoff / 10
    filtered = input

    while threshold > 0:
        filtered = lpf(filtered, ncutoff)
        filtered[::us_rate, ::us_rate] = input[::us_rate, ::us_rate]

        threshold -= 1

    return filtered

In [ ]:
m = ZED_V//LiDAR_V + 1

pg_frame_init = zed_depth
pg_frame_init[::m, :] = vlp_depth

In [ ]:
plt.imshow(pg_frame_init)
plt.show()

In [ ]:
start = time.time()
pg_frame = pg(cp.asarray(pg_frame_init), m, ncutoff=1, threshold=50)
print(time.time() - start)

In [ ]:
plt.imshow(pg_frame.get())
plt.show()